In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/monitor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [139]:
# Функция, определяющая, является ли строка пустой
def isBlank (string):
    if string and string.strip():
        return False
    return True


In [149]:
import re

patterns = "[A-Za-z0-9(){}–«»,:;\"'.“”!#$%&<=>?@\xa0/№-]+"

texts = []

# В цикле читаются все документы из папки ./documents, из текста каждого документа убираются спец. символы,
# указанные в переменной patterns
# После этого тексты всех документов собирается в один массив
for i in range(1,101):
    path='./documents/out{num}.txt'.format(num=i)
    with open(path,'r') as file:
        text = ''
        for line in file.readlines():
            if not isBlank(line):
                line = re.sub(patterns, ' ', line)
                text += ' ' + line
        text = text.replace('\n', ' ')
        texts.append(text)

# print(texts)

In [142]:
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

# функция для создания инвертировнного индекса
# в цикле идет проход по текстам всех документов, для каждого документа в словарь 
# добавляется или обновляется информация о каждом слове(номер документа и кол-во появлений слова в документе)

def create_index(texts):
    tks_count = {}
    for i in range(0,len(texts)):
        for word in texts[i].split(' '):
            token = word.lower()
            if token and token not in stopwords_ru:
                token = token.strip()
                nor_token = morph.normal_forms(token)[0]
                if len(nor_token) > 1:
                    if nor_token in tks_count.keys():
                        if i+1 in tks_count[nor_token].keys():
                            tks_count[nor_token][i+1] += 1
                        else:
                            tks_count[nor_token][i+1] = 1
                    else:
                        tks_count[nor_token] = {i+1:1}
    return tks_count
                    
                
inv_index = create_index(texts)

In [147]:
import json

# запись инвертированного индекса в файл

with open('./result/inverted_index.txt', 'w') as file:
    for key,value in inv_index.items():
        file.write('{}: {}\n'.format(key, json.dumps(value)))

In [148]:
# чтение файла со ссылками на все использованные документы и запись ссылок в массив

path='./index.txt'
index = []
with open(path,'r') as file:
    text = ''
    for line in file.readlines():
        if not isBlank(line):
            index.append(line.split(' ')[-1].replace('\n', ''))



In [145]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()


# функция поиска
# сначала запрос разбивается на список выражений по оператору +(или)
# после этого каждое выражение разбивается на список слов по оператору *(и)
# для каждого слова находятся все документы, в которых оно встречается
# находится перечесение списков документов для данного выражения
# если перед словом стоит минус, то документы, в которых встречается это слово, вычитаются из пересечения
# списки документов, полученные из списка выражений, складываются в один итоговый список документов


def bool_search(query, index, inv_index):
    docs = []
    exprs = query.split('+')
    for expr in exprs:
        words = expr.split('*')
        tempp = []
        tempm = []
        mi = False
        for word in words:
            word = word.strip()
            if '-' in word:
                word = word.replace('-', '')
                mi = True
            else:
                mi = False
            word = morph.normal_forms(word)[0]
            if word in inv_index.keys():
                curr = list(inv_index[word].keys())
                if mi:
                    if len(tempm) == 0:
                        tempm = curr
                    i = 0
                    while i < len(tempm):
                        t = tempm[i]
                        if t not in curr:
                            tempm.remove(t)
                            i-=1
                        i+=1
                else:
                    if len(tempp) == 0:
                        tempp = curr
                    i = 0
                    while i < len(tempp):
                        t = tempp[i]
                        if t not in curr:
                            tempp.remove(t)
                            i-=1
                        i+=1
        i = 0
        while i < len(tempp):
            tem = tempp[i]
            if tem in tempm:
                tempp.remove(tem)
                i-=1
            i+=1
        for el in tempp:
            if el not in docs:
                docs.append(el)
    print(len(docs))
    print(docs)
    for i in range(0, len(docs)):
        docs[i] = index[docs[i]-1]
        print(docs[i])
        
    return docs
        
        

In [146]:
query = 'чудесный * день + -хороший * человек'

docs = bool_search(query, index, inv_index)

29
[14, 23, 30, 36, 47, 54, 64, 80, 90, 98, 99, 20, 24, 25, 29, 31, 33, 34, 38, 46, 49, 60, 68, 69, 71, 81, 86, 88, 97]
https://panorama.pub/5585-yuzhnoafrikanskie-strausy-vpali-v-depressiyu-posle-vstrechi-s-pervoj-ledi-frantsii.html
https://panorama.pub/5464-v-kitae-administratsiya-goroda.html
https://panorama.pub/5369-britanskie-uchyonye-rasskazali-ob-opasnosti-mytya-golovy-shampunem.html
https://panorama.pub/5335-porno-altajsk.html
https://panorama.pub/5259-v-velikobritanii-poyavilis-shkoly-s-obyazatelnym-nosheniem-hidzhaba-dlya-devochek.html
https://panorama.pub/5211-gosdepartament-ssha-priznal.html
https://panorama.pub/5149-podrostki-izlechilis-vodkoy.html
https://panorama.pub/4727-pisatel-sorokin-vozmetsya-za-detskuyu-literaturu.html
https://panorama.pub/4972-afroamerikanets-podal-v-sud-iz-za-belyh-golubej-na-svadbe.html
https://panorama.pub/4854-v-avstralii-rabotniki-zooparka-podbrasyvali-krolikov-pokidayushhim-stranu-inostrantsam.html
https://panorama.pub/4893-nashli-granyonnyj